## 3. Building a Feed Forward Neural Network with PyTorch (GPU)

Now, let's make it live on the GPU (if available) to take advantage of all that calculating power!
2 things **must** be on the GPU:
* model
* Variables

### Steps
* Step 1: Load Dataset
* Step 2: Make Dataset Iterable
* Step 3: Create Model Class
* **Step 4: Instantiate Model Class**
* Step 5: Instantiate Loss Class
* Step 6: Instantiate Optimizer Class
* **Step 7: Train Model!**

In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable

In [3]:
'''
Step 1: Load the dataset
'''

train_dataset = dsets.MNIST(root="./data",
                            train=True,
                            transform=transforms.ToTensor(), 
                            download=True)

test_dataset = dsets.MNIST(root='./data',
                           train=False,
                           transform=transforms.ToTensor())

'''
Step 2: Make dataset iterable
'''

batch_size = 100
n_iters = 3000
num_epochs = int(n_iters / (len(train_dataset) / batch_size))  #Return as type INT

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

'''
Step 3: Create Model Class (relu)
'''

class FeedForwardNeuralNetModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(FeedForwardNeuralNetModel, self).__init__()
        #Linear Function 1: 785 --> 100
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        #Non-Linearity 1
        self.relu1 = nn.ReLU()
        
        #Linear Function 2: 100 --> 100
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        #Non-Linearity 2
        self.relu2 = nn.ReLU()
        
        #Linear Function 3: 100 --> 100
        self.fc3 = nn.Linear(hidden_dim, hidden_dim)
        #Non-Linearity 3
        self.relu3 = nn.ReLU()
        
        #Linear Function 4: 100 --> 10 (readout)
        self.fc4 = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        #Linear Function 1
        out = self.fc1(x)
        #Non-Linearity 1
        out = self.relu1(out)
        
        #Linear Function 2
        out = self.fc2(out)
        #Non-Linearity 2
        out = self.relu2(out)
        
        #Linear Function 3
        out = self.fc3(out)
        #Non-Linearity 3
        out = self.relu3(out)
        
        #Linear Func. 4 (readout)
        out = self.fc4(out)
        return out

'''
Step 4: Instantiate Model Class
'''
input_dim = 28*28
hidden_dim = 100
output_dim = 10

model = FeedForwardNeuralNetModel(input_dim, hidden_dim, output_dim)

#####################
# Use GPU for Model #
#####################

if torch.cuda.is_available():
    model.cuda()
'''
Step 5: Instantiate Loss Class
'''

criterion = nn.CrossEntropyLoss()

'''
Step 6: Instantiate Optimizer Class
'''

learning_rate = 0.1
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

'''
Step 7: Train the ReLU model
'''
iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Load images as Variable
        #####################
        # Use GPU for Model #
        #####################
        if torch.cuda.is_available():
            images = Variable(images.view(-1,28*28).cuda())
            labels = Variable(labels.cuda())
        else:
            images = Variable(images.view(-1,28*28))
            labels = Variable(labels)
        
        #Clear gradients w.r.t. parameters
        optimizer.zero_grad()
        
        #Forward pass to get outputs / logits
        outputs = model(images)
        
        #Calculate Loss: softmax --> Cross Entropy Loss
        loss = criterion(outputs, labels)
        
        #Get gradients w.r.t. parameters
        loss.backward()
        
        #Update parameters
        optimizer.step()
        
        iter += 1
        if iter % 500 == 0:
            # Calculate Accuracy
            correct = 0 
            total = 0
            #Iterate through the test dataset
            for images, labels in test_loader:
                # Load images to Torch Variable
                #####################
                # Use GPU for Model #
                #####################
                if torch.cuda.is_available():
                    images = Variable(images.view(-1,28*28).cuda())
                else:
                    images = Variable(images.view(-1,28*28))
                
                # Forward pass only to get outputs/logits
                outputs = model(images)
                
                # Get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)
                
                # Total number of labels
                total += labels.size(0)
                
                #####################
                # Use GPU for Model #
                #####################
                # Total correct predictions
                if torch.cuda.is_available():
                    correct += (predicted.cpu() == labels.cpu()).sum() #must bring predicted varial
                else:
                    correct += (predicted == labels).sum()
                    
            accuracy = 100 * correct / total
            
            # Print Loss
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.data[0], accuracy))

Iteration: 500. Loss: 0.3954589068889618. Accuracy: 90.9
Iteration: 1000. Loss: 0.20050807297229767. Accuracy: 94.08
Iteration: 1500. Loss: 0.11840119957923889. Accuracy: 95.57
Iteration: 2000. Loss: 0.1011844277381897. Accuracy: 96.44
Iteration: 2500. Loss: 0.11257284879684448. Accuracy: 96.73
Iteration: 3000. Loss: 0.043644145131111145. Accuracy: 96.72


### Wooo! That section went really well. It was really helpful for learning the nuts and bolts with activation functions and what they do. I also got a good feel for the basic steps involved in creating a basic neural network model in PyTorch. Then, we learned a little bit about how to add layers to the model, and the impact that has. Closed with a 3 hidden layer feedforward neural network using ReLU activation. Finally, added the necessary code to allow the model to run on GPU if it exists. 

## Now on to the big one - Convolutional Neural Networks! 